<a href="https://colab.research.google.com/github/Ishita95-harvad/FIDE-Google-Efficient-Chess-AI-Challenge/blob/main/chess_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FIDE-Google Efficient Chess AI Challenge

This notebook implements a resource-efficient chess AI agent for the [FIDE & Google Kaggle competition](https://www.kaggle.com/competitions/fide-google-efficiency-chess-ai-challenge).

## Competition Constraints
- 5 MiB RAM limit
- Single 2.20GHz CPU core
- 64KiB compressed submission size limit
- 10s with 0.1s Simple Delay time control

## Install Dependencies
First, let's install the required packages:

In [1]:
!pip install python-chess==1.999 numpy==1.24.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 55.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 28.7 MB/s eta 0:00:00
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=7f22a57e827d1f515af920875d8b83bd364974cbb6b5caacc2c4d84ce0dba719
  Stored in directory: /root/.cache/pip/wheels/fb/5d/5c/59a62d8a695285e59ec9c1f66add6f8a9ac4152499a2be0113
Successfully built chess
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
pymc 5.22.0 requires numpy>=1.25.0, but you have numpy 1.24.3 which is incompatible.
albucore 0.0.24 requires nu

## Import Required Libraries

In [1]:
import chess
import numpy as np
import time
from typing import Dict, List, Optional, Tuple

## Chess Agent Implementation

Our chess agent implements:
1. Minimax search with alpha-beta pruning
2. Basic evaluation function
3. Iterative deepening for time management

In [2]:
class ChessAgent:
    def __init__(self):
        # Piece values for basic material evaluation
        self.piece_values = {
            chess.PAWN: 100,
            chess.KNIGHT: 320,
            chess.BISHOP: 330,
            chess.ROOK: 500,
            chess.QUEEN: 900,
            chess.KING: 20000
        }

        # Simple piece-square tables for positional evaluation
        self.pst = self._initialize_piece_square_tables()

    def _initialize_piece_square_tables(self) -> Dict:
        """Initialize basic piece-square tables for positional evaluation."""
        # Simple piece-square tables (to be optimized)
        pst = {
            chess.PAWN: np.array([
                0,  0,  0,  0,  0,  0,  0,  0,
                50, 50, 50, 50, 50, 50, 50, 50,
                10, 10, 20, 30, 30, 20, 10, 10,
                5,  5, 10, 25, 25, 10,  5,  5,
                0,  0,  0, 20, 20,  0,  0,  0,
                5, -5,-10,  0,  0,-10, -5,  5,
                5, 10, 10,-20,-20, 10, 10,  5,
                0,  0,  0,  0,  0,  0,  0,  0
            ]),
            chess.KNIGHT: np.array([
                -50,-40,-30,-30,-30,-30,-40,-50,
                -40,-20,  0,  0,  0,  0,-20,-40,
                -30,  0, 10, 15, 15, 10,  0,-30,
                -30,  5, 15, 20, 20, 15,  5,-30,
                -30,  0, 15, 20, 20, 15,  0,-30,
                -30,  5, 10, 15, 15, 10,  5,-30,
                -40,-20,  0,  5,  5,  0,-20,-40,
                -50,-40,-30,-30,-30,-30,-40,-50
            ])
        }
        return pst

    def evaluate_position(self, board: chess.Board) -> float:
        """
        Evaluate the current position.
        Returns a score from white's perspective.
        """
        if board.is_checkmate():
            return -20000 if board.turn else 20000

        score = 0

        # Material evaluation
        for piece_type in self.piece_values:
            score += len(board.pieces(piece_type, chess.WHITE)) * self.piece_values[piece_type]
            score -= len(board.pieces(piece_type, chess.BLACK)) * self.piece_values[piece_type]

        # Basic mobility evaluation
        score += len(list(board.legal_moves)) * (1 if board.turn else -1)

        return score

    def get_best_move(self, board: chess.Board, depth: int = 3) -> Optional[chess.Move]:
        """Find the best move using minimax with alpha-beta pruning."""
        def minimax(board: chess.Board, depth: int, alpha: float, beta: float, maximizing: bool) -> Tuple[float, Optional[chess.Move]]:
            if depth == 0 or board.is_game_over():
                return self.evaluate_position(board), None

            best_move = None
            if maximizing:
                max_eval = float('-inf')
                for move in board.legal_moves:
                    board.push(move)
                    eval_score, _ = minimax(board, depth - 1, alpha, beta, False)
                    board.pop()

                    if eval_score > max_eval:
                        max_eval = eval_score
                        best_move = move
                    alpha = max(alpha, eval_score)
                    if beta <= alpha:
                        break
                return max_eval, best_move
            else:
                min_eval = float('inf')
                for move in board.legal_moves:
                    board.push(move)
                    eval_score, _ = minimax(board, depth - 1, alpha, beta, True)
                    board.pop()

                    if eval_score < min_eval:
                        min_eval = eval_score
                        best_move = move
                    beta = min(beta, eval_score)
                    if beta <= alpha:
                        break
                return min_eval, best_move

        _, best_move = minimax(board, depth, float('-inf'), float('inf'), True)
        return best_move

## Competition Entry Point

This is the main function that will be called by the competition framework:

In [3]:
def agent(obs, config):
    """
    Main agent function that will be called by the competition framework.
    Args:
        obs: Observation from the environment
        config: Configuration for the game
    Returns:
        move: A chess move in UCI format (e.g., 'e2e4')
    """
    # Initialize board from FEN if provided
    board = chess.Board(obs.get('fen', chess.STARTING_FEN))

    # Create agent instance
    chess_agent = ChessAgent()

    # Get the best move with iterative deepening
    start_time = time.time()
    time_limit = 0.1  # Conservative time limit to ensure we don't timeout
    depth = 1
    best_move = None

    while time.time() - start_time < time_limit and depth <= 4:
        try:
            move = chess_agent.get_best_move(board, depth)
            if move:
                best_move = move
            depth += 1
        except Exception:
            break

    # Return the best move found
    return best_move.uci() if best_move else list(board.legal_moves)[0].uci()

## Test the Agent

Let's test our agent with a simple game position:

In [4]:
# Test from starting position
test_obs = {'fen': chess.STARTING_FEN}
test_config = {}

# Get move from agent
move = agent(test_obs, test_config)
print(f"Agent's move: {move}")

# Display the board
board = chess.Board()
print("\nStarting position:")
print(board)

Agent's move: e2e3

Starting position:
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


**Install Dependencies:**

Run the following cell to install python-chess and numpy

In [5]:
!pip install python-chess numpy

**Run the cell to define the ChessAI class and agent function**.

In [9]:
import chess
import numpy as np
import time

# Piece-Square Tables for positional evaluation (simplified for efficiency)
PIECE_SQUARE_TABLES = {
    chess.PAWN: np.array([
        0,  0,  0,  0,  0,  0,  0,  0,
        5, 10, 10, -20, -20, 10, 10,  5,
        5, -5, -10,  0,  0, -10, -5,  5,
        0,  0,  0, 20, 20,  0,  0,  0,
        5,  5, 10, 25, 25, 10,  5,  5,
        10, 10, 20, 30, 30, 20, 10, 10,
        50, 50, 50, 50, 50, 50, 50, 50,
        0,  0,  0,  0,  0,  0,  0,  0
    ], dtype=np.int8),
    chess.KNIGHT: np.array([
        -50, -40, -30, -30, -30, -30, -40, -50,
        -40, -20,   0,   5,   5,   0, -20, -40,
        -30,   5,  10,  15,  15,  10,   5, -30,
        -30,   0,  15,  20,  20,  15,   0, -30,
        -30,   5,  15,  20,  20,  15,   5, -30,
        -30,   0,  10,  15,  15,  10,   0, -30,
        -40, -20,   0,   0,   0,   0, -20, -40,
        -50, -40, -30, -30, -30, -30, -40, -50
    ], dtype=np.int8),
    # Simplified: Use the same table for other pieces to save memory
}

# Material values
PIECE_VALUES = {
    chess.PAWN: 100,
    chess.KNIGHT: 320,
    chess.BISHOP: 330,
    chess.ROOK: 500,
    chess.QUEEN: 900,
    chess.KING: 20000
}

class ChessAI:
    def __init__(self, time_limit=9.5):  # Conservative time limit (10s - 0.5s buffer)
        self.time_limit = time_limit
        self.start_time = None
        self.nodes_searched = 0

    def should_stop(self):
        if self.start_time is None:
            return False
        return (time.time() - self.start_time) >= self.time_limit

    def evaluate(self, board):
        if board.is_checkmate():
            return -9999 if board.turn == chess.WHITE else 9999
        if board.is_stalemate() or board.is_insufficient_material():
            return 0

        # Material balance
        material = 0
        for piece_type in PIECE_VALUES:
            material += len(board.pieces(piece_type, chess.WHITE)) * PIECE_VALUES[piece_type]
            material -= len(board.pieces(piece_type, chess.BLACK)) * PIECE_VALUES[piece_type]

        # Positional evaluation using piece-square tables
        positional = 0
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece is None:
                continue
            sign = 1 if piece.color == chess.WHITE else -1
            piece_type = piece.piece_type
            # Flip the square for Black's perspective
            idx = square if piece.color == chess.WHITE else chess.SQUARE_NAMES.index(chess.square_name(square)[0] + str(9 - int(chess.square_name(square)[1])))
            table = PIECE_SQUARE_TABLES.get(piece_type, PIECE_SQUARE_TABLES[chess.PAWN])
            positional += sign * table[idx]

        # Mobility evaluation
        mobility = len(list(board.legal_moves)) * 2
        board.turn = not board.turn
        mobility -= len(list(board.legal_moves)) * 2
        board.turn = not board.turn

        return material + positional + mobility

    def alpha_beta(self, board, depth, alpha, beta):
        self.nodes_searched += 1
        if depth == 0 or board.is_game_over() or self.should_stop():
            return self.evaluate(board)

        best_value = -float('inf') if board.turn == chess.WHITE else float('inf')
        for move in board.legal_moves:
            board.push(move)
            value = self.alpha_beta(board, depth - 1, alpha, beta)
            board.pop()
            if board.turn == chess.WHITE:
                best_value = max(best_value, value)
                alpha = max(alpha, best_value)
            else:
                best_value = min(best_value, value)
                beta = min(beta, best_value)
            if beta <= alpha:
                break
        return best_value

    def iterative_deepening(self, board):
        self.start_time = time.time()
        self.nodes_searched = 0
        best_move = None
        depth = 1
        while not self.should_stop():
            best_value = -float('inf') if board.turn == chess.WHITE else float('inf')
            alpha, beta = -float('inf'), float('inf')
            moves = list(board.legal_moves)
            if not moves:
                break
            for move in moves:
                board.push(move)
                value = self.alpha_beta(board, depth - 1, alpha, beta)
                board.pop()
                if board.turn == chess.WHITE:
                    if value > best_value:
                        best_value = value
                        best_move = move
                    alpha = max(alpha, best_value)
                else:
                    if value < best_value:
                        best_value = value
                        best_move = move
                    beta = min(beta, best_value)
            depth += 1
        return best_move

def agent(observation, configuration):
    board = chess.Board(observation['fen'])
    ai = ChessAI()
    move = ai.iterative_deepening(board)
    return str(move) if move else "resign"

if __name__ == "__main__":
    # Test the agent with a starting position
    board = chess.Board()
    ai = ChessAI()
    move = ai.iterative_deepening(board)
    print(f"Recommended move: {move}")

Recommended move: d2d3


In [7]:
# Test the ChessAgent directly
board = chess.Board()
ai = ChessAgent() # Corrected class name
move = ai.get_best_move(board, depth=3) # Call get_best_move with a depth
print(f"Recommended move: {move}")

Recommended move: e2e3


**Verify Resource Usage**

In [8]:
import psutil
print(psutil.Process().memory_info().rss / 1024 / 1024, "MiB")

106.37109375 MiB


In [10]:
%%writefile submission.py
import chess
import numpy as np
import time

# Piece-Square Tables (simplified for memory efficiency)
PIECE_SQUARE_TABLES = {
    chess.PAWN: np.array([
        0,  0,  0,  0,  0,  0,  0,  0,
        5, 10, 10, -20, -20, 10, 10,  5,
        5, -5, -10,  0,  0, -10, -5,  5,
        0,  0,  0, 20, 20,  0,  0,  0,
        5,  5, 10, 25, 25, 10,  5,  5,
        10, 10, 20, 30, 30, 20, 10, 10,
        50, 50, 50, 50, 50, 50, 50, 50,
        0,  0,  0,  0,  0,  0,  0,  0
    ], dtype=np.int8),
    chess.KNIGHT: np.array([
        -50, -40, -30, -30, -30, -30, -40, -50,
        -40, -20,   0,   5,   5,   0, -20, -40,
        -30,   5,  10,  15,  15,  10,   5, -30,
        -30,   0,  15,  20,  20,  15,   0, -30,
        -30,   5,  15,  20,  20,  15,   5, -30,
        -30,   0,  10,  15,  15,  10,   0, -30,
        -40, -20,   0,   0,   0,   0, -20, -40,
        -50, -40, -30, -30, -30, -30, -40, -50
    ], dtype=np.int8),
}

# Material values
PIECE_VALUES = {
    chess.PAWN: 100,
    chess.KNIGHT: 320,
    chess.BISHOP: 330,
    chess.ROOK: 500,
    chess.QUEEN: 900,
    chess.KING: 20000
}

class ChessAI:
    def __init__(self, time_limit=9.5):  # 10s - 0.5s buffer
        self.time_limit = time_limit
        self.start_time = None
        self.nodes_searched = 0

    def should_stop(self):
        if self.start_time is None:
            return False
        return (time.time() - self.start_time) >= self.time_limit

    def evaluate(self, board):
        if board.is_checkmate():
            return -9999 if board.turn == chess.WHITE else 9999
        if board.is_stalemate() or board.is_insufficient_material():
            return 0

        # Material balance
        material = 0
        for piece_type in PIECE_VALUES:
            material += len(board.pieces(piece_type, chess.WHITE)) * PIECE_VALUES[piece_type]
            material -= len(board.pieces(piece_type, chess.BLACK)) * PIECE_VALUES[piece_type]

        # Positional evaluation
        positional = 0
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece is None:
                continue
            sign = 1 if piece.color == chess.WHITE else -1
            piece_type = piece.piece_type
            idx = square if piece.color == chess.WHITE else chess.SQUARE_NAMES.index(chess.square_name(square)[0] + str(9 - int(chess.square_name(square)[1])))
            table = PIECE_SQUARE_TABLES.get(piece_type, PIECE_SQUARE_TABLES[chess.PAWN])
            positional += sign * table[idx]

        # Mobility evaluation
        mobility = len(list(board.legal_moves)) * 2
        board.turn = not board.turn
        mobility -= len(list(board.legal_moves)) * 2
        board.turn = not board.turn

        return material + positional + mobility

    def alpha_beta(self, board, depth, alpha, beta):
        self.nodes_searched += 1
        if depth == 0 or board.is_game_over() or self.should_stop():
            return self.evaluate(board)

        best_value = -float('inf') if board.turn == chess.WHITE else float('inf')
        for move in board.legal_moves:
            board.push(move)
            # Prioritize checkmate detection
            if board.is_checkmate():
                board.pop()
                return -9999 if board.turn == chess.WHITE else 9999
            value = self.alpha_beta(board, depth - 1, alpha, beta)
            board.pop()
            if board.turn == chess.WHITE:
                best_value = max(best_value, value)
                alpha = max(alpha, best_value)
            else:
                best_value = min(best_value, value)
                beta = min(beta, best_value)
            if beta <= alpha:
                break
        return best_value

    def iterative_deepening(self, board):
        self.start_time = time.time()
        self.nodes_searched = 0
        best_move = None
        depth = 1
        while not self.should_stop():
            best_value = -float('inf') if board.turn == chess.WHITE else float('inf')
            alpha, beta = -float('inf'), float('inf')
            moves = list(board.legal_moves)
            if not moves:
                break
            for move in moves:
                board.push(move)
                if board.is_checkmate():
                    board.pop()
                    return move
                value = self.alpha_beta(board, depth - 1, alpha, beta)
                board

.pop()
                if board.turn == chess.WHITE:
                    if value > best_value:
                        best_value = value
                        best_move = move
                    alpha = max(alpha, best_value)
                else:
                    if value < best_value:
                        best_value = value
                        best_move = move
                    beta = min(beta, best_value)
            depth += 1
        return best_move

def agent(observation, configuration):
    board = chess.Board(observation['fen'])
    ai = ChessAI()
    move = ai.iterative_deepening(board)
    return str(move) if move else "resign"

if __name__ == "__main__":
    # Test with the given position
    fen = "3rk1K1/1p1p1pp1/1PpPpP1p/4q1n1/1PpPp3/8/8/1K1N4 w - - 0 1"
    board = chess.Board(fen)
    ai = ChessAI(time_limit=2.0)  # Short time limit for testing
    move = ai.iterative_deepening(board)
    print(f"Recommended move: {move}")

Writing submission.py


In [11]:
!zip submission.zip submission.py
!ls -lh submission.zip

  adding: submission.py (deflated 72%)
-rw-r--r-- 1 root root 1.7K May 29 04:16 submission.zip
